In [7]:
import pandas as pd
import pyodbc
import sqlalchemy as sa
import psycopg2 as p
import os
import array
import string
import json

In [8]:
# path to folder with year of files
year = r'C:\Users\mmgri\Desktop\2005'

# creating database connection string
conn = p.connect("host=capstonealaindexing2018.postgres.database.azure.com dbname=mmgtest user=hstandeffer@capstonealaindexing2018 password=Alaindexing!")
cur = conn.cursor()

#declare array for dataframes
dfs = []

#iterate thru each week
for week in os.listdir(year):
        #get name and path
        weekName = os.fsdecode(week)
        weekPath = os.path.join(year, weekName)
        
        #iterate thru files in week
        for flat in os.listdir(weekPath):
            
            flatName = os.fsdecode(flat)
            flatPath = os.path.join(weekPath, flatName)

            #finding flattened folder
            string = 'flattened'
            
            if flatName == string:

                for game in os.listdir(flatPath):

                    gameName = os.fsdecode(game)
                    gamePath = os.path.join(flatPath, gameName)
                        
                    print(gameName)

                    # find all json files
                    if gameName.endswith(".json"): 

                        # read json file to dataframe
                        df = pd.read_json(os.path.join(gamePath), orient='columns')

                        # add dataframe to list of dataframes
                        dfs.append(df)
                        continue
                    else:
                        continue

# combine list into one dataframe
df = pd.concat(dfs)

# remove commas from descriptions to avoid erros
df['description'] = df['description'].str.replace('[^\w\s]', '')

# normalize dataframes 
dfGame = df[['awayAbbr', 'awayId', 'awayTeam', 'gameId', 'homeAbbr', 'homeId', 'homeTeam', 'week', 'year']]
dfPlay = df[['awayScore', 'clock', 'defenseAbbr', 'defenseId', 'defenseTeam', 'description', 'distance', 'down', 
             'driveIndex', 'endYardLine', 'gameId', 'homeScore', 'offenseAbbr', 'offenseId', 
             'offenseTeam', 'playIndex', 'quarter', 'type', 'yardLine', 'yardsGained']]

# remove duplicates from game
dfGame.drop_duplicates(subset ="gameId", keep = 'first', inplace = True) 

# array for file names and table names 
fileNames = ['game', 'play']

# array for dormalized dataframes
dfNormalized = [dfGame, dfPlay]

# loop to add files to postgres database
for df, file in zip(dfNormalized, fileNames):
    print(df.shape)
    print(file)
    
    # writes dataframe to a csv 
    df.to_csv(r'C:\Users\mmgri\Desktop\csv\\' + file + '.csv', sep=',', index=False)
    fileString = r'C:\Users\mmgri\Desktop\csv\\' + file + '.csv'
    
    # loads csv to database 
    # with open(fileString, 'r') as f:
      #  next(f)
       # cur = conn.cursor()
        #cur.copy_from(f, file, sep=',')
        #conn.commit()
     


213520036 - Colorado State vs North Texas.json
213530276 - Marshall vs East Carolina.json
213540152 - NC State vs Pitt.json
213590254 - Utah vs USC.json
213610024 - Stanford vs Georgia Tech.json
213610066 - Iowa State vs Alabama.json
213620061 - Georgia vs Boston College.json
213620245 - Texas A_M vs TCU.json
213620251 - Texas vs Washington.json
213632306 - Kansas State vs Syracuse.json
213632641 - Texas Tech vs Iowa.json
213632649 - Toledo vs Cincinnati.json
213650002 - Auburn vs North Carolina.json
213650252 - BYU vs Louisville.json
213650265 - Washington St vs Purdue.json
213650278 - Fresno State vs Michigan State.json
213652348 - LA Tech vs Clemson.json
220010201 - Oklahoma vs Arkansas.json
220010259 - Virginia Tech vs Florida State.json
220010356 - Illinois vs LSU.json
220012483 - Oregon vs Colorado.json
220012579 - South Carolina vs Ohio State.json
220012633 - Tennessee vs Michigan.json
220020057 - Florida vs Maryland.json
220032390 - Miami vs Nebraska.json
212370201 - Oklahoma v

212440221 - Pitt vs ETSU.json
212440228 - Clemson vs UCF.json
212440245 - Texas A_M vs McNeese.json
212440251 - Texas vs New Mexico St.json
212440258 - Virginia vs Richmond.json
212440259 - Virginia Tech vs UConn.json
212440333 - Alabama vs UCLA.json
212442005 - Air Force vs Oklahoma.json
212442294 - Iowa vs Kent State.json
212442305 - Kansas vs Missouri State.json
212442483 - Oregon vs Wisconsin.json
212442572 - Southern Miss vs Oklahoma State.json
212442579 - South Carolina vs Boise State.json
212442633 - Tennessee vs Syracuse.json
212450278 - Fresno State vs Oregon State.json
212452132 - Cincinnati vs Purdue.json
212460344 - Miss St vs Memphis.json
212490152 - NC State vs Indiana.json
212492751 - Wyoming vs Texas A_M.json
212502439 - UNLV vs Northwestern.json
212510002 - Auburn vs Ole Miss.json
212510008 - Arkansas vs Tennessee.json
212510009 - Arizona State vs San Diego State.json
212510012 - Arizona vs Idaho.json
212510024 - Stanford vs Boston College.json
212510025 - Cal vs BYU.j

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(423, 9)
game
(83083, 20)
play
